# Assembler

In [1]:
def preprocess_assembly_code(assembly_code):
    assembly_code = assembly_code.splitlines()

    line_no=0
    line_alias={}

    #Line alias
    for line in assembly_code:
        s = line.strip().split(' ')
        if not s[0]:
            continue
        if s[0][-1]==':':
            line_alias[s[0][0:-1]]=line_no

        line_no+=1
    # print("line_alias:   ",line_alias, '\n')

    #Preprocessing (Removing alias)
    preprocessd_assembly_code = []
    for line in assembly_code:
        s = line.strip().split(' ')
        if not s[0]:
            continue
        if s[0][-1]==':':
            s=s[1:]
        # print(s)        

        inst = s[0]
        if inst == 'IF_P' or inst == 'ELSE_P':     #IF_P or ELSE_P
            out= [inst , line_alias[s[1]] ]
        else:
            out = s

        preprocessd_assembly_code.append(out)

    return preprocessd_assembly_code

# preprocess_assembly_code(assembly_code)

In [2]:
class SPCore():
    def __init__(self, THRD_ID):
        # self.R = [0 for i in range(16)]
        self.R={
            'R0'    :0,'R1'    :0,'R2'    :0,'R3'    :0,'R4'    :0,'R5'    :0,'R6'    :0,'R7'    :0,                'R8'    :0,'R9'    :0,'R10'    :0,'R11'    :0,'R12'    :0,'R13'    :0,'R14'    :0,
            'R15'    :0
        }
        self.THRD_ID = THRD_ID
        self.P = True
        self.P_stack = [True]

    def __str__(self):
        return (
            "SPCore:\t"+str(self.THRD_ID)+'\n'+
            "P\t"+str(self.P)+'\n'+
            "P_stack\t"+str(self.P_stack)+'\n'+
            str(self.R)+'\n'
        )


    def pushMask(self):
        self.P_stack.append(self.P)
    def popMask(self):
        self.P_stack.pop()
    def curMask(self):
        return self.P_stack[-1]
    def invMask(self):
        self.P_stack[-1] = not self.P_stack[-1]


##############################
def all_p_false():
    for core in cores:
        if core.curMask()==True:
            return False
    return True

def all_p_true():
    for core in cores:
        if core.curMask()==False:
            return False
    return True

def valid(cores):
    return [core for core in cores if core.curMask()==True ]

def pushStack():
    for core in cores:
        core.pushMask()

def compStack():
    for core in cores:
        core.invMask()

def popStack():
    for core in cores:
        core.popMask()    

In [4]:
class Scheduler():
    def __init__(self):
        self.assembly_code = open("test.as", "r").read()
        self.assembly_code = preprocess_assembly_code(self.assembly_code)
        print(self.assembly_code, '\n')
        # print(len(self.assembly_code), '\n')
        self.PC = 0

    def runProgram(self):        
        while self.PC <len(self.assembly_code):
            self.execute(self.assembly_code[self.PC])
            print('PC',self.PC)
            print(cores[0])
        print('\n____Finished___')

    def execute(self, s):
        # print(s)
        opcode = s[0]
        print(s)
        print('valid',len(valid(cores)))

        if opcode == 'LOAD':
            x=s[1]
            y=s[2]
            # self.R[x]=DataMem[self.R[y]]
            for core in valid(cores):
                core.R[x]=DataMem[core.R[y]] #TODO check mask
        elif opcode == 'LOADI':
            x=s[1]
            I=int(s[2])
            # self.R[x]=I
            for core in valid(cores):
                core.R[x]=I
        elif opcode == 'LOADC':
            x=s[1]
            c=s[2]
            if c=='THRD_ID': #TODO:add rest
                # self.R[x]=self.THRD_ID
                for core in valid(cores):
                    core.R[x]=core.THRD_ID
        elif opcode == 'STORE':
            x=s[1]
            y=s[2]
            # DataMem[self.R[y]] = self.R[x]
            for core in valid(cores):
                DataMem[core.R[y]] = core.R[x]
        elif opcode == 'SETP':
            op = s[1]
            x = s[2] 
            y = s[3]
            if op=='EQ': #TODO: Add rest
                for core in valid(cores):
                    core.P = (core.R[x]==core.R[y])

        elif opcode == 'IF_P':
            L = int(s[1])
            pushStack()
            print("all_p_false",all_p_false())
            if all_p_false():
                self.PC = L
                return


        elif opcode == 'ELSE_P':
            L = int(s[1])
            compStack()
            if all_p_true():
                self.PC = L
                return

        elif opcode == 'ENDIF':
            popStack()

        else:
            raise Exception("Unknown Instruction")


        self.PC+=1
        return

In [5]:
cores = [
    SPCore(0), SPCore(1), SPCore(2), SPCore(3)
]

gpu = Scheduler()

DataMem = [5,1,5,1]
gpu.runProgram()

[['LOADC', 'R0', 'THRD_ID'], ['LOAD', 'R5', 'R0'], ['LOADI', 'R6', '5'], ['SETP', 'EQ', 'R5', 'R6'], ['IF_P', 7], ['LOADI', 'R5', '10'], ['ELSE_P', 8], ['LOADI', 'R5', '20'], ['ENDIF'], ['STORE', 'R5', 'R0']] 

['LOADC', 'R0', 'THRD_ID']
valid 4
PC 1
SPCore:	0
P	True
P_stack	[True]
{'R0': 0, 'R1': 0, 'R2': 0, 'R3': 0, 'R4': 0, 'R5': 0, 'R6': 0, 'R7': 0, 'R8': 0, 'R9': 0, 'R10': 0, 'R11': 0, 'R12': 0, 'R13': 0, 'R14': 0, 'R15': 0}

['LOAD', 'R5', 'R0']
valid 4
PC 2
SPCore:	0
P	True
P_stack	[True]
{'R0': 0, 'R1': 0, 'R2': 0, 'R3': 0, 'R4': 0, 'R5': 5, 'R6': 0, 'R7': 0, 'R8': 0, 'R9': 0, 'R10': 0, 'R11': 0, 'R12': 0, 'R13': 0, 'R14': 0, 'R15': 0}

['LOADI', 'R6', '5']
valid 4
PC 3
SPCore:	0
P	True
P_stack	[True]
{'R0': 0, 'R1': 0, 'R2': 0, 'R3': 0, 'R4': 0, 'R5': 5, 'R6': 5, 'R7': 0, 'R8': 0, 'R9': 0, 'R10': 0, 'R11': 0, 'R12': 0, 'R13': 0, 'R14': 0, 'R15': 0}

['SETP', 'EQ', 'R5', 'R6']
valid 4
PC 4
SPCore:	0
P	True
P_stack	[True]
{'R0': 0, 'R1': 0, 'R2': 0, 'R3': 0, 'R4': 0, 'R5': 5, 'R

In [6]:
DataMem

[10, 20, 10, 20]

# tinyGPU

In [ ]:
# class ALU():
#     def __init__(self):
#         self.Z=0
#         self.Out=0

#     def aluc(self, aluc_signal, A,B,C=None):
#         if aluc_signal == ALU_CLEAR:
#             self.Out = 0
#             self.Z = 0
#         elif aluc_signal == ALU_INC:
#             # TODO
#             pass
#         # TODO:Add the rest

In [37]:
# class IFU():
#     def __init__(self):
#         self.PC = 0
#         self.IR = 0
#         self.TIR = 0
#     def incPC(self):
#         self.PC+=1
#     def loadFromI(self):
#         self.PC=self.TIR

In [ ]:
# class CU_Microsequencer(): #OR State Machine


#     #State Fetch

#     #State Decode

#     #State Execute
    

In [38]:
# class TinyGPU():
#     def __init__(self):
#         self.IFU= IFU()


#         self.core = [
#             Core(), Core()
#         ]


#     def mask_reset(self):
#         # TODO
#         pass
#     def mask_invert(self):
#         # TODO
#         pass

In [39]:
tinyGPU = TinyGPU()

In [42]:
tinyGPU.IFU.incPC()
tinyGPU.IFU.PC

2

In [41]:
tinyGPU.core[0].R[0]

0

In [31]:
# class InstMemory():
#     def __init__(self):
#         #TODO
#         pass
#     def loadFromAssembly(self, assembly_code):
#         #TODO
#         pass
    
# class DataMemory():
#     def __init__(self):
#         # TODO
#         pass
#     def loadFromList(self, ls):
#         # TODO
#         pass